# Crawling Berita

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse, urljoin

def dapatkan_kategori_berita():
    """
    Fungsi untuk mengambil daftar semua kategori berita dari menu navigasi
    website bangsaonline.com.
    """
    print("Mencari kategori berita di bangsaonline.com...")
    kategori_list = {}
    url_home = "https://bangsaonline.com/"
    try:
        response = requests.get(url_home, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        nav_menu = soup.select_one('ul#nav')
        if not nav_menu:
            print("Menu navigasi (ul#nav) tidak ditemukan.")
            return {}

        for item in nav_menu.find_all("a"):
            href = item.get("href")
            nama_kategori = item.get_text(strip=True)
            
            if href and nama_kategori:
                path_parts = urlparse(href).path.strip("/").split("/")
                
                if len(path_parts) == 2 and path_parts[0] == 'kanal':
                    url_lengkap = urljoin(url_home, href)
                    if nama_kategori not in kategori_list:
                        kategori_list[nama_kategori] = url_lengkap

        print(f"Ditemukan {len(kategori_list)} kategori berita valid.")
        
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar kategori berita: {e}")
        
    return kategori_list

def scrape_semua_berita():
    """
    Fungsi utama untuk melakukan scraping berita dari semua kategori yang ditemukan.
    """
    daftar_kategori = dapatkan_kategori_berita()

    if not daftar_kategori:
        print("Tidak ada kategori yang bisa di-scrape. Program berhenti.")
        return

    data_berita = []
    scraped_links = set()
    url_home = "https://bangsaonline.com/"

    for nama_kategori, url_kategori in daftar_kategori.items():
        print(f"\n--- Scraping Kategori: {nama_kategori.upper()} ---")
        artikel_diambil = 0
        
        try:
            response_kategori = requests.get(url_kategori, timeout=10)
            response_kategori.raise_for_status()
            soup_kategori = BeautifulSoup(response_kategori.text, "html.parser")

            # --- SELECTOR DIKEMBALIKAN SESUAI PERMINTAAN ---
            list_artikel = soup_kategori.select("h3.entry-title a")
            
            if not list_artikel:
                print("Tidak ada tautan artikel ditemukan di halaman ini.")
                continue

            for artikel in list_artikel:
                # Anda bisa mengubah angka 2 ini jika ingin scrape lebih banyak per kategori
                if artikel_diambil >= 2:
                    break
                
                link_parsial = artikel.get("href")
                if not link_parsial:
                    continue
                
                link = urljoin(url_home, link_parsial)
                
                if link in scraped_links:
                    continue

                scraped_links.add(link)

                try:
                    resp_detail = requests.get(link, timeout=10)
                    resp_detail.raise_for_status()
                    soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

                    # --- SELECTOR DIKEMBALIKAN SESUAI PERMINTAAN ---
                    judul_element = soup_detail.select_one("h1.entry-title")
                    konten_berita = soup_detail.select_one("div.post")
                    
                    if judul_element and konten_berita:
                        judul = judul_element.get_text(strip=True)
                        
                        id_berita = None
                        try:
                            path_parts = urlparse(link).path.strip("/").split("/")
                            if len(path_parts) > 1 and path_parts[1].isdigit():
                                id_berita = path_parts[1]
                        except (IndexError, AttributeError):
                            id_berita = None
                        
                        for unwanted in konten_berita.select("div.baca-juga"):
                            unwanted.decompose()
                        
                        paragraf = [p.get_text(strip=True) for p in konten_berita.select("p")]
                        isi = " ".join(paragraf)

                        if isi:
                            data_berita.append({
                                "id_berita": id_berita,
                                "kategori": nama_kategori,
                                "judul": judul,
                                "isi_berita": isi,
                                "link": link
                            })
                            artikel_diambil += 1
                            print(f"({artikel_diambil}/2) Berhasil scrape: {judul[:60]}...")
                    
                    time.sleep(1)

                except requests.exceptions.RequestException as e:
                    print(f"  -> Gagal mengambil detail dari {link}: {e}")

        except requests.exceptions.RequestException as e:
            print(f"Gagal memproses halaman kategori {url_kategori}: {e}")
            
    if not data_berita:
        print("\nTidak ada berita yang berhasil di-scrape.")
        return

    df = pd.DataFrame(data_berita)
    df = df[["id_berita", "kategori", "judul", "isi_berita", "link"]]
    
    df.to_csv("hasil_scraping_berita_bangsaonline.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ Proses scraping selesai. {len(df)} berita disimpan ke 'hasil_scraping_berita_bangsaonline.csv'")
    
    return df

# --- Untuk Menjalankan Seluruh Proses Scraping ---
if __name__ == "__main__":
    df_hasil = scrape_semua_berita()
    if df_hasil is not None:
        pd.set_option('display.max_colwidth', 100)
        print("\nContoh hasil data:")

Mencari kategori berita di bangsaonline.com...


Ditemukan 37 kategori berita valid.

--- Scraping Kategori: JATIM ---


(1/2) Berhasil scrape: Festival Kopi Quds Royal Hotel Surabaya Meriahkan Akhir Tahu...


(2/2) Berhasil scrape: PT Freeport Gelar Pelatihan Apartemen Kepiting sebagai Solus...



--- Scraping Kategori: JATIM METRO ---


...

(1/2) Berhasil scrape: Bupati Pamekasan Lantik 4.161 P3K Paruh Waktu, Harap Kerja K...


(2/2) Berhasil scrape: Pemkab Pamekasan Selamatkan 86 Ribu Warga dari Ancaman Putus...



--- Scraping Kategori: NASIONAL ---


(1/2) Berhasil scrape: Kiai Asep Terkejut Dapat Penghargaan dari KPAI sebagai Tokoh...


KeyboardInterrupt: 

In [4]:
df_hasil.head()

,id_berita,kategori,judul,isi_berita,link
0,152149,Jatim,"Judi Sabung Ayam di Pamekasan Digerebek, 6 Orang Diamankan","PAMEKASAN, BANGSAONLINE.com- Satreskrim Polres Pamekasan menggerebek arena judi sabung ayam di D...",https://bangsaonline.com/berita/152149/judi-sabung-ayam-di-pamekasan-digerebek-6-orang-diamankan
1,152148,Jatim,Pemilik Ladang Ganja di Blitar Akui Jual Ganja Kering Rp5 Juta Perkilogram,"BLITAR, BANGSAONLINE.com- Pemilik ladang ganja, SA (38), warga Desa Krisik, Kecamatan Gandusari,...",https://bangsaonline.com/berita/152148/pemilik-ladang-ganja-di-blitar-akui-jual-ganja-kering-rp5...
2,152132,Jatim Metro,"Permudah Warga Urus Surat Secara Digital, Pemdes Banjarsari Luncurkan Aplikasi PAOD","SIDOARJO, BANGSAONLINE.com- Pemerintah Desa (Pemdes) Banjarsari, Kecamatan Buduran, melakukan te...",https://bangsaonline.com/berita/152132/permudah-warga-urus-surat-secara-digital-pemdes-banjarsar...
3,152118,Jatim Metro,"Fesyar 2025 Siap Digelar di Surabaya, Gubernur Khofifah Optimistis Jatim Jadi Pusat Ekonomi Syariah","SURABAYA, BANGSAONLINE.com- Gubernur Khofifah menyatakan optimisme tinggi terhadap penyelenggara...",https://bangsaonline.com/berita/152118/fesyar-2025-siap-digelar-di-surabaya-gubernur-khofifah-op...
4,152142,Jatim Tengah,Wali Kota Kediri dan Staf Ahli Bidang Ekonomi Maritim Kemenko Pangan Tinjau Koperasi Merah Putih,"KOTA KEDIRI, BANGSAONLINE.com- Wali Kota Kediri, Vinanda Prameswati, bersama Staf Ahli Bidang Ek...",https://bangsaonline.com/berita/152142/wali-kota-kediri-dan-staf-ahli-bidang-ekonomi-maritim-kem...
